# Capítulo 6, ejercicio propuesto 8. Deducción del gradiente y del laplaciano en coordenadas esféricas

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Diego Andrés Alvarez, <daalvarez@unal.edu.co>  | Octubre 3, 2023      | Primer código |
|Diego Andrés Alvarez, <daalvarez@unal.edu.co>  | Agosto 13, 2024      | Mejorando comentarios|

$$
\newcommand{\ve}[1]{{\boldsymbol{#1}}}
\newcommand{\ma}[1]{{\boldsymbol{#1}}}
\newcommand{\hati}{\ve{\hat i}}
\newcommand{\hatj}{\ve{\hat j}}
\newcommand{\hatk}{\ve{\hat k}}
\newcommand{\hatr}{\ve{\hat r}}
\newcommand{\hatt}{\ve{\hat \theta}}
\newcommand{\hatp}{\ve{\hat \varphi}}
$$

In [1]:
from sympy import init_printing, latex, symbols, sin, cos, diff, simplify, Function, Matrix, trigsimp,  expand

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

In [2]:
r, t, p = symbols('r, theta, varphi')

Se definen las variables rectangulares $(x, y, z)$ en función de las variables esféricas $(r, \theta, \phi)$:

In [3]:
x = r*sin(t)*cos(p)
y = r*sin(t)*sin(p)
z = r*cos(t)

Se calcula la matriz jacobiana de la transformación $(r, \theta, \varphi) \mapsto (x,y,z):$

In [4]:
dxyz_drtp = Matrix([[diff(x,r),  diff(x,t),  diff(x,p)],
                    [diff(y,r),  diff(y,t),  diff(y,p)],
                    [diff(z,r),  diff(z,t),  diff(z,p)]])

imprimir(r"\frac{\partial(x,y,z)}{\partial(r,\theta,\varphi)} = ", dxyz_drtp)

<IPython.core.display.Math object>

Para calcular las derivadas utilizamos el [teorema de la función inversa](https://es.wikipedia.org/wiki/Matriz_y_determinante_jacobianos); según este, la matriz inversa de la matriz Jacobiana de una función invertible es la matriz Jacobiana de la función inversa:
$$
\underbrace{\begin{pmatrix}
    \frac{\partial r}{\partial x}       & \frac{\partial r}{\partial y}       &  \frac{\partial r}{\partial z}\\[5pt]
    \frac{\partial \theta}{\partial x}  & \frac{\partial \theta}{\partial y}  & \frac{\partial \theta}{\partial z} \\[5pt]
    \frac{\partial \varphi}{\partial x} & \frac{\partial \varphi}{\partial y} & \frac{\partial \varphi}{\partial z}
\end{pmatrix}}_{\frac{\partial(r,\theta,\varphi)}{\partial(x,y,z)}}
=
\underbrace{\begin{pmatrix}
        \frac{\partial x}{\partial r}  & \frac{\partial x}{\partial \theta}  & \frac{\partial x}{\partial \varphi} \\[5pt]
        \frac{\partial y}{\partial r}  & \frac{\partial y}{\partial \theta}  & \frac{\partial y}{\partial \varphi} \\[5pt]
        \frac{\partial z}{\partial r}  & \frac{\partial z}{\partial \theta}  & \frac{\partial z}{\partial \varphi}
\end{pmatrix}^{-1}}_{\frac{\partial(x,y,z)}{\partial(r,\theta,\varphi)}^{-1}}
$$

In [5]:
# se invierte la matriz Jacobiana
drtp_dxyz = simplify(dxyz_drtp**-1)

imprimir(r"\frac{\partial(r,\theta,\varphi)}{\partial(x,y,z)} = ", drtp_dxyz)

<IPython.core.display.Math object>

Se extraen las derivadas de la ecuación anterior:

In [6]:
dr_dx, dr_dy, dr_dz = drtp_dxyz[0,:]
dt_dx, dt_dy, dt_dz = drtp_dxyz[1,:]
dp_dx, dp_dy, dp_dz = drtp_dxyz[2,:]

Se especifican "manualmente" las primeras derivadas utilizando la regla de la cadena para derivar una función $f(r,\theta,\varphi)$ teniendo en cuenta que $r$, $\theta$ y $\varphi$ son funciones de $x$, $y$ y $z$:
\begin{align*}
\frac{\partial f}{\partial x} &= \frac{\partial f}{\partial r}\frac{\partial r}{\partial x} + \frac{\partial f}{\partial \theta} \frac{\partial \theta}{\partial x} + \frac{\partial f}{\partial \varphi} \frac{\partial \varphi}{\partial x}\\
\frac{\partial f}{\partial y} &= \frac{\partial f}{\partial r}\frac{\partial r}{\partial y} + \frac{\partial f}{\partial \theta} \frac{\partial \theta}{\partial y} + \frac{\partial f}{\partial \varphi} \frac{\partial \varphi}{\partial y} \\
\frac{\partial f}{\partial z} &= \frac{\partial f}{\partial r}\frac{\partial r}{\partial z} + \frac{\partial f}{\partial \theta} \frac{\partial \theta}{\partial z} + \frac{\partial f}{\partial \varphi} \frac{\partial \varphi}{\partial z}
\end{align*}    

In [7]:
d_dx = lambda f: diff(f,r)*dr_dx + diff(f,t)*dt_dx + diff(f,p)*dp_dx
d_dy = lambda f: diff(f,r)*dr_dy + diff(f,t)*dt_dy + diff(f,p)*dp_dy
d_dz = lambda f: diff(f,r)*dr_dz + diff(f,t)*dt_dz + diff(f,p)*dp_dz

Se definen las segundas derivadas:

In [8]:
d2_dx2 = lambda f: d_dx(d_dx(f))
d2_dy2 = lambda f: d_dy(d_dy(f))
d2_dz2 = lambda f: d_dz(d_dz(f))

Se definen los vectores unitarios:
\begin{align*}
    \ve{\hat{r}}       &= \sin\theta \cos\varphi \hati + \sin\theta \sin\varphi \hatj + \cos\theta \hatk \\
    \ve{\hat{\theta}}  &= \cos\theta \cos\varphi \hati + \cos\theta \sin\varphi \hatj - \sin\theta \hatk \\
    \ve{\hat{\varphi}} &= -\sin\varphi \hati + \cos \varphi \hatj.
\end{align*}

In [9]:
rg = Matrix([ sin(t)*cos(p), sin(t)*sin(p),  cos(t) ])
tg = Matrix([ cos(t)*cos(p), cos(t)*sin(p), -sin(t) ])
pg = Matrix([       -sin(p),        cos(p),       0 ])

Y luego se define la matriz de transformación como $\ma{T} = \left[\ve{\hat{r}},\ \ve{\hat{\theta}},\ \ve{\hat{\varphi}}  \right]$

In [10]:
T = Matrix.hstack(rg, tg, pg)

imprimir(r"\boldsymbol{T} = ", T)

<IPython.core.display.Math object>

In [11]:
# se especifica que f es función de r, t y p
f = Function('f')(r, t, p)

## Caĺculo del laplaciano en coordenadas esféricas

Se especifica el laplaciano en coordenadas rectangulares:

In [12]:
lapl1 = simplify(d2_dx2(f) + d2_dy2(f) + d2_dz2(f))

Se imprime el laplaciano en coordenadas esféricas:

In [13]:
imprimir(r"\nabla^2 \phi(r, \theta) = ", lapl1)

<IPython.core.display.Math object>

y se compara contra la fórmula que está en el libro:
$$
\nabla^2 f = \frac{1}{r^2}\frac{\partial}{\partial r}\left(r^2 \frac{\partial f}{\partial r}\right) + \frac{1}{r^2\sin\theta}\frac{\partial}{\partial \theta}\left(\sin\theta \frac{\partial f}{\partial \theta}\right) + \frac{1}{r^2\sin^2\theta}\frac{\partial^2 f}{\partial \varphi^2}
$$

In [14]:
lapl2 = (1/(r**2))*diff(r**2 * diff(f,r), r) + (1/(r**2 * sin(t)))*diff(sin(t) * diff(f,t), t) + (1/(r**2 * sin(t)**2))*diff(f,p,2)

Si `lapl1 - lapl2 == 0` es porque ambas expresiones son matemáticamente equivalentes:

In [15]:
simplify(lapl1 - lapl2)

## Cálculo del gradiente en coordenadas esféricas:

Se especifica el gradiente en coordenadas rectangulares, el cual está definido con respecto a la base $\left\{\hati,\ \hatj, \hatk \right\}$:

In [16]:
grad_f_ijk = Matrix([ d_dx(f),
                      d_dy(f),
                      d_dz(f)  ])

Se convierte el gradiente de la base $\left\{\hati,\ \hatj, \hatk \right\}$ a la base $\left\{\hatr,\ \hatt, \hatp \right\}$:

In [17]:
grad_f_rtp = trigsimp(T.T * grad_f_ijk)

Se imprime el gradiente en coordenadas esféricas:

In [18]:
imprimir(r"\boldsymbol{\nabla} f(r, \theta, \varphi) = ", grad_f_rtp)

<IPython.core.display.Math object>